## OME-Zarr notebook

This notebook will demonstrate the `qim3d` functionalities that specifically target the OME-Zarr file format.

OME-Zarr is a cloud-native, multi-dimensional file format optimized for storing and analyzing large-scale volumetric imaging data. It leverages the Zarr storage framework, which organizes data in a chunked, hierarchical structure, enabling efficient random access to specific regions without loading entire datasets into memory. The format supports multi-resolution storage, allowing visualization and analysis at different levels of detail, improving performance for large datasets. OME-Zarr stores metadata in JSON format following the Open Microscopy Environment (OME) model, ensuring interoperability across a wide range of bioimaging tools. Additionally, its compatibility with modern data science libraries like Dask and Xarray allows for scalable parallel processing, making it an ideal choice for applications in microscopy, medical imaging, and machine learning.

First we will fetch a large data file, which we can save to the OME-Zarr file type. Here we will use the `io.Downloader` class. First we define the variable, and then we evaluate what data we would like to download, by visualizing the options with the 'help' command.

In [49]:
import qim3d

downloader = qim3d.io.Downloader()
help(downloader)


Help on Downloader in module qim3d.io._downloader object:

class Downloader(builtins.object)
 |  Class for downloading large data files available on the [QIM data repository](https://data.qim.dk/).
 |  
 |  Attributes:
 |      folder_name (str or os.PathLike): Folder class with the name of the folder in <https://data.qim.dk/>
 |  
 |  Syntax for downloading and loading a file is `qim3d.io.Downloader().{folder_name}.{file_name}(load_file=True)`
 |  
 |  ??? info "Overview of available data"
 |      Below is a table of the available folders and files on the [QIM data repository](https://data.qim.dk/).
 |  
 |      Folder name         | File name                                                                                                          | File size
 |      ------------------- | ------------------------------------------------------------------------------------------------------------------ | ---------------------------------------------
 |      `Coal`              | `CoalBri

After deciding to use the coral data, we fetch the data. We use the 'load_file' argument to load it while downloading it. Additionally, we can use the vizualisation tool `slicer_orthogonal` too explore the volume from three different axes.

In [50]:
vol = downloader.Corals.Coral2_DOWNSAMPLED(load_file=True)

qim3d.viz.slicer_orthogonal(vol, color_map="magma")

File already downloaded:
/home/s214735/qim3d/docs/notebooks/Corals/Coral2_DOWNSAMPLED.tif

Loading Coral2_DOWNSAMPLED.tif


Loading:   0%|          | 0.00B/153MB  [00:00<?, ?B/s]

Loaded shape: (500, 400, 400)
Using virtual stack


Attempting to visualize the volume in a three-dimensional space will require a lot of computing power due to the size of the volume. However using the OME-Zarr data type, we can visualize it in chunks. Additionally we can save it in the .zarr format.

First we export the file using the `export_ome_zarr` method.

In [81]:
qim3d.io.export_ome_zarr(
    'coral.zarr',
    vol,
    chunk_size=200,
    downsample_rate=2,
    replace=True)

Exporting data to OME-Zarr format at coral.zarr
Number of scales: 3
Calculating the multi-scale pyramid
- Scale 0: (500, 400, 400)
- Scale 1: (250, 200, 200)
- Scale 2: (125, 100, 100)
Writing data to disk


Saving:   0%|          | 0.00/166 [00:00<?, ?Chunks/s]


All done!


We can then use the `chunks` method to visualize the chunks from the volume. Here we have both options for exploring the volume slices or the entire 3D object.

In [52]:
qim3d.viz.chunks('coral.zarr')

After looking at the object, we see that some of it is obstructed by background noise. Therefore we attempt to remove that with a threshold, followed by another export and visualization of the volume:

In [82]:
vol_t = vol > 30000

qim3d.io.export_ome_zarr(
    'coral_threshold.zarr',
    vol_t,
    chunk_size=200,
    downsample_rate=2,
    replace=True)

qim3d.viz.chunks('coral_threshold.zarr')

Exporting data to OME-Zarr format at coral_threshold.zarr
Number of scales: 3
Calculating the multi-scale pyramid
- Scale 0: (500, 400, 400)
- Scale 1: (250, 200, 200)
- Scale 2: (125, 100, 100)
Writing data to disk


Saving:   0%|          | 0.00/83.0 [00:00<?, ?Chunks/s]


All done!


For next time the volume is to be used, we can import it easily using the `import_ome_zarr` method:

In [83]:
vol_t = qim3d.io.import_ome_zarr('coral_threshold.zarr')

Data contains 3 scales:
- Scale 0: (500, 400, 400)
- Scale 1: (250, 200, 200)
- Scale 2: (125, 100, 100)

Loading scale 0 with shape (500, 400, 400)
